# Quick peak fitting while on Raman
- This folder is for quick peak fitting while on the Raman 
- It doesnt require you to run the Ne line file - it just uses a guess of the average Ne line of your system
- This is very useful when looking at new samples so you know what sort of densities you are working with, so can adjust laser power etc. 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import DiadFit as pf
pf.__version__

## Specifying filetype and folder

In [ ]:
import os
DayFolder=os.getcwd()
meta_path=DayFolder 
spectra_path=DayFolder + '\spectra'
# Filetype and extension for spectra
spectra_filetype='headless_txt'
spectra_file_ext='.txt'
# file extension for spectra
meta_file_ext='.txt'
# Does your file start with a prefix? E.g 01 Ne_line.txt?
prefix=False
# If so - what is the character separating the prefix from the real name
prefix_str=' '
# Does your instrument have TruPower (WITEC)
TruPower=False
# Save settings to a file to use in all other notebooks
pf.save_settings(meta_path, spectra_path, spectra_filetype, prefix, prefix_str, spectra_file_ext, meta_file_ext, TruPower)



## Grab all files

In [ ]:
exclude_str=['Ne', 'NE', 'Si', 'nodiad', 'Spectrum', 'CRR', 'no_diad', 'settings']
Diad_Files=pf.get_files(path=spectra_path, file_ext=spectra_file_ext, exclude_str=exclude_str)
print(Diad_Files)

### Lets plot the diad to have a look at it

In [ ]:
i=0
plot1=pf.plot_diad(path=spectra_path, filename=Diad_Files[i], filetype=spectra_filetype)

## Get approximate peak positoins

In [ ]:


diad_id_config=pf.diad_id_config(prominence=12, width=1)
diad_id_config

fit_params, data_y_all=pf.loop_approx_diad_fits(spectra_path=spectra_path, config=diad_id_config, 
                       Diad_Files=Diad_Files, filetype=spectra_filetype, plot_figure=False)
fit_params.head()

## Fit Diad 1
- Tweak the parameters in the config files for each group. E.g. how many peaks (fit_peaks), the background positions, the sigma of the diad, and whether or not you want a gaussian background

In [ ]:
model_name='PseudoVoigtModel'
diad1_fit_config_init=pf.diad1_fit_config(
model_name=model_name, fit_peaks=2,
N_poly_bck_diad1=2, lower_bck_diad1=(1180, 1240),
upper_bck_diad1=(1315, 1350),
diad_sigma=0.6,
x_range_residual=10, x_range_baseline=30, 
    y_range_baseline=100, 
HB_prom=fit_params['HB1_abs_prom'].iloc[i], 
diad_prom=fit_params['Diad1_abs_prom'].iloc[i])
diad1_fit_config_init


### See what these fit parameters look like for diad1

In [ ]:
Diad1_fit=pf.fit_diad_1_w_bck(config1=diad1_fit_config_init,
config2=diad_id_config,
path=spectra_path, filename=Diad_Files[i],
filetype=spectra_filetype, plot_figure=True, close_figure=False,
Diad_pos=fit_params['Diad1_pos'].iloc[i],
HB_pos=fit_params['HB1_pos'].iloc[i])
Diad1_fit

In [ ]:
diad1_fit_config2=diad1_fit_config_init

## Fit diad2

In [ ]:

diad2_fit_config_init=pf.diad2_fit_config(model_name=model_name,
fit_peaks=3, fit_gauss=True, gauss_amp= 2*fit_params['HB2_abs_prom'].iloc[i],
lower_bck_diad2=(1310, 1350), diad_sigma=1,  N_poly_bck_diad2=2,                                       
x_range_residual=30, y_range_baseline=100,  
x_range_baseline=30,
HB_prom=fit_params['HB2_abs_prom'].iloc[i], 
diad_prom=fit_params['Diad2_abs_prom'].iloc[i],
C13_prom=fit_params['C13_abs_prom'].iloc[i])
diad2_fit_config_init



### See what these fit parameters look like for diad2

In [ ]:
Diad2_fit=pf.fit_diad_2_w_bck(config1=diad2_fit_config_init,
    config2=diad_id_config,
path=spectra_path, filename=Diad_Files[i], filetype=spectra_filetype,
plot_figure=True, close_figure=False, 
Diad_pos=fit_params['Diad2_pos'].iloc[i],
HB_pos=fit_params['HB2_pos'].iloc[i], 
C13_pos=fit_params['C13_pos'].iloc[i])
Diad2_fit.to_clipboard(excel=True)
Diad2_fit

## Loop over all the files in the group
- This will loop over all the 

In [ ]:
diad2_fit_config_init
from tqdm import tqdm
plot_figure=False# If False, Means doesnt have to make figures, lot faster. 
close_figure=False # If True, wont show figures in notebook, but will still save them in a folder 
Diad_Files_i=Diad_Files
df_Merge = pd.DataFrame([])
for i in tqdm(range(0, len(Diad_Files))): #
    tqdm.write(f"Processing file: {Diad_Files[i]}")
    
    
    # For diad1, config file like you had in the previous. 
    # Only really used to exclude a range (say your spectra has a known spec)
    diad_id_config=pf.diad_id_config(exclude_range1=[1308, 1309])
    
    # Here, the prominence are taken from the fitting in the last notebook
    diad1_fit_config2.HB_prom=fit_params['HB1_abs_prom'].iloc[i]
    diad1_fit_config2.diad_prom=fit_params['Diad1_abs_prom'].iloc[i]
    diad1_fit_config2.gauss_amp=2*fit_params['HB1_abs_prom'].iloc[i]

    Diad1_fit=pf.fit_diad_1_w_bck(config1=diad1_fit_config2,
    config2=diad_id_config, path=spectra_path, filename=fit_params['filename'].iloc[i],
    filetype=spectra_filetype, plot_figure=plot_figure, close_figure=close_figure,
    Diad_pos=fit_params['Diad1_pos'].iloc[i],
    HB_pos=fit_params['HB1_pos'].iloc[i])

    ## Same for diad2, just also has a C13 peak
    diad2_fit_config_init.HB_prom=fit_params['HB2_abs_prom'].iloc[i]
    diad2_fit_config_init.diad_prom=fit_params['Diad2_abs_prom'].iloc[i]
    diad2_fit_config_init.gauss_amp= 2*fit_params['HB2_abs_prom'].iloc[i]
    diad2_fit_config_init.C13_prom=fit_params['C13_abs_prom'].iloc[i]
    
    
    Diad2_fit=pf.fit_diad_2_w_bck(config1=diad2_fit_config_init,
        config2=diad_id_config,
    path=spectra_path, filename=fit_params['filename'].iloc[i], 
    filetype=spectra_filetype,
    plot_figure=plot_figure, close_figure=close_figure,
    Diad_pos=fit_params['Diad2_pos'].iloc[i],
    HB_pos=fit_params['HB2_pos'].iloc[i], 
    C13_pos=fit_params['C13_pos'].iloc[i])
    
    # This combines the outputs into a single dataframe
    data=pf.combine_diad_outputs(filename=fit_params['filename'].iloc[i], prefix=prefix, 
    Diad1_fit=Diad1_fit, path=spectra_path,                  
    Diad2_fit=Diad2_fit)        
    
 

    df_Merge = pd.concat([df_Merge, data], axis=0).reset_index(drop=True)


In [ ]:
df_Merge

## Lets get some approximate fits

In [ ]:
df_Merge

In [ ]:
# An average of the Ne Correction factor for your instrument
Ne_corr_factor_av=0.9979
Ne_err=2.465495739435719e-06
df_Merge['sec since midnight']=30000

#df_Merge['Corrected_Splitting']=df_Merge['Splitting']*Ne_corr_factor_av

df=pf.calculate_density_ucb(df_combo=df_Merge, Ne_pickle_str=None,
 temp='SupCrit', CI_split=0.67, CI_neon=0.67, pref_Ne=Ne_corr_factor_av, Ne_err=Ne_err)



In [ ]:
df

In [ ]:
## This lets you read the labels
fig, (ax1) = plt.subplots(1, 1, figsize=(14,10))
df=df.loc[df['Density g/cm3']>0]
ax1.errorbar(df['Corrected_Splitting'], df['Density g/cm3'],
xerr=df['Corrected_Splitting_σ'], yerr=df['σ Density g/cm3'],
fmt='o', ecolor='grey', elinewidth=0.8, mfc='red', ms=6, mec='k', capsize=2)
for i, filename in enumerate(df['filename']):
    print(filename)
    ax1.annotate(filename, (df['Corrected_Splitting'].iloc[i], df['Density g/cm3'].iloc[i]), xytext=(5,0),  fontsize=6, textcoords='offset points')

ax1.set_xlabel('Corrected Splitting (cm$^{-1}$)')
ax1.set_ylabel('Density g/cm3')

In [ ]:

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14,10))
df=df.loc[df['Density g/cm3']>0]
ax1.errorbar(df['Corrected_Splitting'], df['Density g/cm3'],
xerr=df['Corrected_Splitting_σ'], yerr=df['σ Density g/cm3'],
fmt='o', ecolor='grey', elinewidth=0.8, mfc='red', ms=6, mec='k', capsize=2)

ax1.set_xlabel('Corrected Splitting (cm$^{-1}$)')
ax1.set_ylabel('Density g/cm3')

ax2.plot(df['Corrected_Splitting'], 
100*df['Corrected_Splitting_σ_Ne']/(df['Corrected_Splitting_σ_Ne']+df['Corrected_Splitting_σ_peak_fit']),
 'ok', mfc='orange')
ax2.set_ylabel('% of splitting error from Neon Correction')
ax2.set_xlabel('Corrected Splitting (cm$^{-1}$)')
ax3.plot(df['Density g/cm3'], 100*df['σ Density g/cm3 (from Ne+peakfit)']/df['Density g/cm3'], 
'ok', mfc='cyan', ms=5, label='Error: Ne + pk fit')


ax3.plot(df['Density g/cm3'], 100*df['σ Density g/cm3 (from densimeter)']/df['Density g/cm3'], 
'ok', mfc='blue', ms=5, label='Error: Densimeter')

ax3.plot(df['Density g/cm3'], 100*df['σ Density g/cm3']/df['Density g/cm3'], 
'ok', mfc='yellow', ms=5, label='Overall Error')
ax3.set_ylabel('% error')
ax3.set_xlabel('Density g/cm3')
ax3.legend()

ax4.plot(df['Density g/cm3'], df['σ Density g/cm3 (from densimeter)'], 
'ok', mfc='blue', ms=5, label='Error: Densimeter')
ax4.plot(df['Density g/cm3'], df['σ Density g/cm3 (from Ne+peakfit)'], 
'ok', mfc='cyan', ms=5, label='Error: Splitting')
ax4.plot(df['Density g/cm3'], df['σ Density g/cm3'], 
'ok', mfc='yellow', ms=5, label='Overall Error')
ax4.set_ylabel('error (g/cm3)')
ax4.set_xlabel('Density g/cm3')
ax4.legend()

fig.tight_layout()

### This plays a sound when the notebook is done if you have a tendency to procrastinate

In [ ]:
# !pip install winotify

from winotify import Notification, audio

toast= Notification(app_id="VSCode",title="Notebook completed",
                    msg="Step3b_Secondary_Peaks is done!",
                    duration="short")
toast.set_audio(audio.Mail,loop=False)
toast.show()